##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Hello, many worlds

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/quantum/tutorials/hello_many_worlds"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">TensorFlow.org에서 보기</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/quantum/blob/master/docs/tutorials/hello_many_worlds.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Google Colab에서 실행</a></td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/quantum/blob/master/docs/tutorials/hello_many_worlds.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">View source on GitHub</a>
  </td>
  <td><a href="https://storage.googleapis.com/tensorflow_docs/quantum/docs/tutorials/hello_many_worlds.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">노트북 다운로드하기</a></td>
</table>

이 튜토리얼에서는 기존 신경망이 큐비트 보정 오류의 수정 방법을 어떤 식으로 학습하는지 알아봅니다. Noisy Intermediate Scale Quantum(NISQ) 회로를 생성, 편집 및 호출하기 위한 Python 프레임워크인 <a target="_blank" href="https://github.com/quantumlib/Cirq" class="external">Cirq</a>를 소개하고 Cirq가 TensorFlow Quantum과 상호 작용하는 방식을 소개합니다.

## 설정

In [ ]:
!pip install tensorflow==2.7.0

TensorFlow Quantum을 설치합니다.

In [ ]:
!pip install tensorflow-quantum

In [ ]:
# Update package resources to account for version changes.
import importlib, pkg_resources
importlib.reload(pkg_resources)

이제 TensorFlow 및 모듈 종속성을 가져옵니다.

In [ ]:
import tensorflow as tf
import tensorflow_quantum as tfq

import cirq
import sympy
import numpy as np

# visualization tools
%matplotlib inline
import matplotlib.pyplot as plt
from cirq.contrib.svg import SVGCircuit

## 1. 기본 사항

### 1.1 Cirq 및 매개변수화된 양자 회로

TensorFlow Quantum(TFQ)을 살펴보기 전에 <a target="_blank" href="https://github.com/quantumlib/Cirq" class="external">Cirq</a>에 관한 몇 가지 기본 사항을 알아보겠습니다. Cirq는 Google의 양자 컴퓨팅을 위한 Python 라이브러리로, 정적 및 매개변수화된 게이트를 포함한 회로를 정의하는 데 사용합니다.

Cirq는 <a target="_blank" href="https://www.sympy.org" class="external">SymPy</a> 기호를 사용하여 자유 매개변수를 나타냅니다.

In [ ]:
a, b = sympy.symbols('a b')

다음은 매개변수를 사용하여 2-큐비트 회로를 생성하는 코드입니다.

In [ ]:
# Create two qubits
q0, q1 = cirq.GridQubit.rect(1, 2)

# Create a circuit on these qubits using the parameters you created above.
circuit = cirq.Circuit(
    cirq.rx(a).on(q0),
    cirq.ry(b).on(q1), cirq.CNOT(control=q0, target=q1))

SVGCircuit(circuit)

회로를 평가하기 위해 `cirq.Simulator` 인터페이스를 사용할 수 있습니다. `cirq.ParamResolver` 객체를 전달하여 회로의 자유 매개변수를 특정 숫자로 바꿉니다. 다음 코드는 매개변수화된 회로의 원시 상태 벡터 출력을 계산합니다.

In [ ]:
# Calculate a state vector with a=0.5 and b=-0.5.
resolver = cirq.ParamResolver({a: 0.5, b: -0.5})
output_state_vector = cirq.Simulator().simulate(circuit, resolver).final_state_vector
output_state_vector

상태 벡터는 시뮬레이션 외부에서 직접 액세스할 수 없습니다(위 출력의 복소수에 주목). 물리적으로 현실적이려면 상태 벡터를 기존 컴퓨터가 이해할 수 있는 실수로 변환하는 측정을 지정해야 합니다. Cirq는 <a target="_blank" href="https://en.wikipedia.org/wiki/Pauli_matrices" class="external">Pauli 연산자</a> $\hat{X}$, $\hat{Y}$ 및 $\hat{Z}$를 조합하여 사용하여 측정을 지정합니다. 그림과 같이, 다음 코드는 방금 시뮬레이션한 상태 벡터에서 $\hat{Z}_0$ 및 $\frac{1}{2}\hat{Z}_0 + \hat{X}_1$를 측정합니다.

In [ ]:
z0 = cirq.Z(q0)

qubit_map={q0: 0, q1: 1}

z0.expectation_from_state_vector(output_state_vector, qubit_map).real

In [ ]:
z0x1 = 0.5 * z0 + cirq.X(q1)

z0x1.expectation_from_state_vector(output_state_vector, qubit_map).real

### 1.2 양자 회로를 텐서로 사용하기

TensorFlow Quantum(TFQ)은 Cirq 객체를 텐서로 변환하는 함수인 `tfq.convert_to_tensor`를 제공합니다. 이를 통해 Cirq 객체를 <a target="_blank" href="https://www.tensorflow.org/quantum/api_docs/python/tfq/layers">양자 레이어</a> 및 <a target="_blank" href="https://www.tensorflow.org/quantum/api_docs/python/tfq/get_expectation_op">양자 ops</a>로 보낼 수 있습니다. 이 함수는 Cirq Circuits 및 Cirq Paulis의 목록 또는 배열에서 호출할 수 있습니다.

In [ ]:
# Rank 1 tensor containing 1 circuit.
circuit_tensor = tfq.convert_to_tensor([circuit])

print(circuit_tensor.shape)
print(circuit_tensor.dtype)

그러면 CIRQ 객체가 `tf.string` 텐서로 인코딩되고, `tfq` 연산은 이 텐서를 필요에 따라 디코딩합니다.

In [ ]:
# Rank 1 tensor containing 2 Pauli operators.
pauli_tensor = tfq.convert_to_tensor([z0, z0x1])
pauli_tensor.shape

### 1.3 회로 시뮬레이션 배치 처리하기

TFQ는 기대값, 샘플 및 상태 벡터를 계산하기 위한 메서드를 제공합니다. 지금은 *기대값*에 중점을 두겠습니다.

기대값을 계산하기 위한 최상위 인터페이스는 `tf.keras.Layer`인 `tfq.layers.Expectation` 레이어입니다. 가장 간단한 형태에서 이 레이어는 많은 `cirq.ParamResolvers`에 걸쳐 매개변수화된 회로를 시뮬레이션하는 것과 동일합니다. 그러나, TFQ는 다음 TensorFlow 의미 체계에 따라 배치 처리를 허용하고 회로는 효율적인 C++ 코드를 사용하여 시뮬레이션됩니다.

`a` 및 `b` 매개변수를 대체할 값의 배치를 만듭니다.

In [ ]:
batch_vals = np.array(np.random.uniform(0, 2 * np.pi, (5, 2)), dtype=np.float32)

Cirq의 매개변수 값에 대한 회로 실행을 배치 처리하려면 루프가 필요합니다.

In [ ]:
cirq_results = []
cirq_simulator = cirq.Simulator()

for vals in batch_vals:
    resolver = cirq.ParamResolver({a: vals[0], b: vals[1]})
    final_state_vector = cirq_simulator.simulate(circuit, resolver).final_state_vector
    cirq_results.append(
        [z0.expectation_from_state_vector(final_state_vector, {
            q0: 0,
            q1: 1
        }).real])

print('cirq batch results: \n {}'.format(np.array(cirq_results)))

TFQ에서는 같은 연산이 단순화됩니다.

In [ ]:
tfq.layers.Expectation()(circuit,
                         symbol_names=[a, b],
                         symbol_values=batch_vals,
                         operators=z0)

## 2. 양자-고전 하이브리드 최적화

기본 사항을 살펴보았으므로 이제 TensorFlow Quantum을 사용하여 *양자-고전 하이브리드 신경망*을 구성해 보겠습니다. 단일 큐비트를 제어하도록 고전적 신경망을 훈련합니다. `0` 또는 `1` 상태에서 큐비트를 올바르게 준비하도록 제어를 최적화하여 시뮬레이션된 계통적 보정 오류를 극복합니다. 아키텍처는 다음 그림과 같습니다.

<img src="./images/nn_control1.png" width="1000">

신경망 없이도 쉽게 해결할 수 있는 문제이지만, 다루고자 하는 주제는 TFQ를 사용하여 해결할 수 있는 실제 양자 제어 문제와 유사합니다. 여기서는 `tf.keras.Model` 내의 `tfq.layers.ControlledPQC`(매개변수화된 양자 회로) 레이어를 사용하는 양자-고전 계산의 엔드 투 엔드 예제를 보여줍니다.

For the implementation of this tutorial, this architecture is split into 3 parts:

- *입력 회로* 또는 *데이터 포인트 회로*: 처음 3개의 $R$ 게이트
- *제어 회로*: 다른 3개의 $R$ 게이트
- *컨트롤러*: 제어 회로의 매개변수를 설정하는 고전적 신경망

### 2.1 제어 회로 정의

위 그림과 같이 학습 가능한 1bit 순환을 정의합니다. 이것은 여기서 다루는 제어 회로에 해당합니다.

In [ ]:
# Parameters that the classical NN will feed values into.
control_params = sympy.symbols('theta_1 theta_2 theta_3')

# Create the parameterized circuit.
qubit = cirq.GridQubit(0, 0)
model_circuit = cirq.Circuit(
    cirq.rz(control_params[0])(qubit),
    cirq.ry(control_params[1])(qubit),
    cirq.rx(control_params[2])(qubit))

SVGCircuit(model_circuit)

### 2.2 컨트롤러

이제 컨트롤러 네트워크를 정의합니다. 

In [ ]:
# The classical neural network layers.
controller = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='elu'),
    tf.keras.layers.Dense(3)
])

명령 배치가 입력되면 컨트롤러는 제어 회로의 제어 신호 배치를 출력합니다.

컨트롤러는 무작위로 초기화되므로 이들 출력은 아직 유용하지 않습니다.

In [ ]:
controller(tf.constant([[0.0],[1.0]])).numpy()

### 2.3 회로에 컨트롤러 연결하기

`tfq`를 사용하여 하나의 `keras.Model`로 제어 회로에 컨트롤러를 연결합니다.

이 스타일의 모델 정의에 대한 자세한 내용은 [Keras 함수형 API 가이드](https://www.tensorflow.org/guide/keras/functional)를 참조하세요.

먼저, 모델에 대한 입력을 정의합니다. 

In [ ]:
# This input is the simulated miscalibration that the model will learn to correct.
circuits_input = tf.keras.Input(shape=(),
                                # The circuit-tensor has dtype `tf.string` 
                                dtype=tf.string,
                                name='circuits_input')

# Commands will be either `0` or `1`, specifying the state to set the qubit to.
commands_input = tf.keras.Input(shape=(1,),
                                dtype=tf.dtypes.float32,
                                name='commands_input')


다음으로, 해당 입력에 연산을 적용하여 계산을 정의합니다.

In [ ]:
dense_2 = controller(commands_input)

# TFQ layer for classically controlled circuits.
expectation_layer = tfq.layers.ControlledPQC(model_circuit,
                                             # Observe Z
                                             operators = cirq.Z(qubit))
expectation = expectation_layer([circuits_input, dense_2])

이제 이 계산을 `tf.keras.Model` 패키지로 구성합니다.

In [ ]:
# The full Keras model is built from our layers.
model = tf.keras.Model(inputs=[circuits_input, commands_input],
                       outputs=expectation)

네트워크 아키텍처는 아래 모델의 플롯으로 표시됩니다. 이 모델 플롯을 아키텍처 다이어그램과 비교하여 정확성을 확인합니다.

참고: 시스템에 `graphviz` 패키지를 설치해야 할 수 있습니다.

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True, dpi=70)

이 모델은 컨트롤러에 대한 명령, 그리고 컨트롤러가 출력을 수정하려는 입력 회로의 두 가지 입력을 받습니다. 

### 2.4 데이터세트

모델은 각 명령에 대해 올바른 측정값 $\hat{Z}$를 출력하려고 시도합니다. 명령과 올바른 값은 아래에 정의되어 있습니다.

In [ ]:
# The command input values to the classical NN.
commands = np.array([[0], [1]], dtype=np.float32)

# The desired Z expectation value at output of quantum circuit.
expected_outputs = np.array([[1], [-1]], dtype=np.float32)

이것이 이 작업의 전체 훈련 데이터세트는 아닙니다. 데이터세트의 각 데이터 포인트에는 입력 회로도 필요합니다.

### 2.4 입력 회로 정의

아래의 입력 회로는 모델이 수정 방법을 학습할 임의의 오보정을 정의합니다.

In [ ]:
random_rotations = np.random.uniform(0, 2 * np.pi, 3)
noisy_preparation = cirq.Circuit(
  cirq.rx(random_rotations[0])(qubit),
  cirq.ry(random_rotations[1])(qubit),
  cirq.rz(random_rotations[2])(qubit)
)
datapoint_circuits = tfq.convert_to_tensor([
  noisy_preparation
] * 2)  # Make two copied of this circuit

각 데이터 포인트에 하나씩 두 개의 회로 복사본이 있습니다.

In [ ]:
datapoint_circuits.shape

### 2.5 훈련

정의된 입력으로 `tfq` 모델 실행을 테스트할 수 있습니다.

In [ ]:
model([datapoint_circuits, commands]).numpy()

이제 표준 훈련 프로세스를 실행하여 이들 값을 `expected_outputs`에 맞게 조정합니다.

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.05)
loss = tf.keras.losses.MeanSquaredError()
model.compile(optimizer=optimizer, loss=loss)
history = model.fit(x=[datapoint_circuits, commands],
                    y=expected_outputs,
                    epochs=30,
                    verbose=0)

In [ ]:
plt.plot(history.history['loss'])
plt.title("Learning to Control a Qubit")
plt.xlabel("Iterations")
plt.ylabel("Error in Control")
plt.show()

이 플롯에서 신경망이 계통적인 오보정을 극복하는 방법을 학습했음을 알 수 있습니다.

### 2.6 출력 확인하기

이제 훈련된 모델을 사용하여 큐비트 보정 오류를 수정합니다. Cirq 사용:

In [ ]:
def check_error(command_values, desired_values):
  """Based on the value in `command_value` see how well you could prepare
  the full circuit to have `desired_value` when taking expectation w.r.t. Z."""
  params_to_prepare_output = controller(command_values).numpy()
  full_circuit = noisy_preparation + model_circuit

  # Test how well you can prepare a state to get expectation the expectation
  # value in `desired_values`
  for index in [0, 1]:
    state = cirq_simulator.simulate(
        full_circuit,
        {s:v for (s,v) in zip(control_params, params_to_prepare_output[index])}
    ).final_state_vector
    expt = cirq.Z(qubit).expectation_from_state_vector(state, {qubit: 0}).real
    print(f'For a desired output (expectation) of {desired_values[index]} with'
          f' noisy preparation, the controller\nnetwork found the following '
          f'values for theta: {params_to_prepare_output[index]}\nWhich gives an'
          f' actual expectation of: {expt}\n')


check_error(commands, expected_outputs)

훈련 중 손실 함수의 값으로부터 모델의 학습 효과를 대략적으로 짐작할 수 있습니다. 손실이 낮을수록 위 셀의 기대값이 `desired_values`에 가깝습니다. 매개변수 값에 관심이 없다면 `tfq`를 사용하여 위의 출력을 확인할 수 있습니다.

In [ ]:
model([datapoint_circuits, commands])

## 3 여러 연산자의 고유 상태를 준비하도록 학습하기

1과 0에 해당하는 $\pm \hat{Z}$ 고유 상태의 선택은 임의적이었습니다. 간단히 1이 $+ \hat{Z}$ 고유 상태에 해당하고 0이 $-\hat{X}$ 고유 상태에 해당하기를 바랬을 것입니다. 이를 수행하는 한 가지 방법은 아래 그림과 같이 각 명령에 다른 측정 연산자를 지정하는 것입니다.

<img src="./images/nn_control2.png" width="1000">

이를 위해서는 <code>tfq.layers.Expectation</code>을 사용해야 합니다. 이제 회로, 명령 및 연산자의 세 가지 객체를 포함하도록 입력이 커졌습니다. 출력은 여전히 기대값입니다.

### 3.1 새로운 모델 정의

이 작업을 수행하는 모델을 살펴보겠습니다.

In [ ]:
# Define inputs.
commands_input = tf.keras.layers.Input(shape=(1),
                                       dtype=tf.dtypes.float32,
                                       name='commands_input')
circuits_input = tf.keras.Input(shape=(),
                                # The circuit-tensor has dtype `tf.string` 
                                dtype=tf.dtypes.string,
                                name='circuits_input')
operators_input = tf.keras.Input(shape=(1,),
                                 dtype=tf.dtypes.string,
                                 name='operators_input')

다음은 컨트롤러 네트워크입니다.

In [ ]:
# Define classical NN.
controller = tf.keras.Sequential([
    tf.keras.layers.Dense(10, activation='elu'),
    tf.keras.layers.Dense(3)
])

`tfq`를 사용하여 회로와 컨트롤러를 하나의 `keras.Model`로 결합합니다.

In [ ]:
dense_2 = controller(commands_input)

# Since you aren't using a PQC or ControlledPQC you must append
# your model circuit onto the datapoint circuit tensor manually.
full_circuit = tfq.layers.AddCircuit()(circuits_input, append=model_circuit)
expectation_output = tfq.layers.Expectation()(full_circuit,
                                              symbol_names=control_params,
                                              symbol_values=dense_2,
                                              operators=operators_input)

# Contruct your Keras model.
two_axis_control_model = tf.keras.Model(
    inputs=[circuits_input, commands_input, operators_input],
    outputs=[expectation_output])

### 3.2 데이터세트

이제 `model_circuit`에 제공하는 각 데이터 포인트에 대해 측정하려는 연산자도 포함합니다.

In [ ]:
# The operators to measure, for each command.
operator_data = tfq.convert_to_tensor([[cirq.X(qubit)], [cirq.Z(qubit)]])

# The command input values to the classical NN.
commands = np.array([[0], [1]], dtype=np.float32)

# The desired expectation value at output of quantum circuit.
expected_outputs = np.array([[1], [-1]], dtype=np.float32)

### 3.3 훈련

이제 새로운 입력 및 출력이 준비되었으므로 keras를 사용하여 다시 한 번 훈련할 수 있습니다.

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.05)
loss = tf.keras.losses.MeanSquaredError()

two_axis_control_model.compile(optimizer=optimizer, loss=loss)

history = two_axis_control_model.fit(
    x=[datapoint_circuits, commands, operator_data],
    y=expected_outputs,
    epochs=30,
    verbose=1)

In [ ]:
plt.plot(history.history['loss'])
plt.title("Learning to Control a Qubit")
plt.xlabel("Iterations")
plt.ylabel("Error in Control")
plt.show()

손실 함수가 0으로 감소했습니다.

`controller`는 독립형 모델로 이용할 수 있습니다. 컨트롤러를 호출하고 각 명령 신호에 대한 응답을 확인하세요. 이들 출력을 `random_rotations`의 내용과 올바르게 비교하려면 약간의 작업이 필요합니다.

In [ ]:
controller.predict(np.array([0,1]))

성공: 첫 번째 모델의 `check_error` 함수가 이 새 모델 아키텍처에서 동작하도록 조정할 수 있는지 확인해 보세요.